In [1]:
import bs4
import requests
import pandas
import IPython

In [13]:
links = [open('france 1 of 5.html').read(), 
         requests.get('https://tools.cofrac.fr/en/easysearch?list-73223214').content, 
         requests.get('https://tools.cofrac.fr/en/easysearch?list-6529721').content, 
         requests.get('https://tools.cofrac.fr/en/easysearch?list-67668472').content,
         requests.get('https://tools.cofrac.fr/en/easysearch?list-24236605').content]

In [14]:
def getRawRecords(page):
    soup = bs4.BeautifulSoup(page)

    allDivs = soup.findAll('div', class_=['entite', 'themas'])

    recordsRaw = []

    for div in allDivs:
        if div.get('class') == ['entite']:
            recordsRaw.append({'Main Info': div,
                               'Files': []})
        else:
            recordsRaw[-1]['Files'].append(div)
    
    return recordsRaw

getRawRecords(links[0])

[{'Main Info': <div class="entite" id="enti_31112714">
  <h2><div class="topleft">1 / 39</div><div class="topright">search by Name</div><div style="clear:both;"></div>AO SGS Vostok Limited		</h2>
  <b>AO SGS Vostok Limited</b><br/>
  	Building 7 18 Andropova Prospekt |  MOSCOW  115432 | RUSSIAN FEDERATION<br/>
  	Tel. : +7495 775 44 55 | Fax : +7495 775 44 50<br/>
  	Type : <font color="#004494"><b>Client</b></font> | Activity : <font color="#004494"><b>
  	Inspection</b></font><br/>
  </div>,
  'Files': [<div class="themas"><div style="float:left;width:27%;"><div class="numdoss"><a class="toggleButton domaine_plus" href="javascript:void(0);"><span>[+]</span></a><span> File 3-1504</span> </div><div class="anx"><div class="small-margins" style="border: 1px solid white;"><a href="/annexes/sect3/3-1504.pdf" target="_blank">Accreditation certificate</a>*</div></div></div><div class="themas_right" style="float:right;width:72%;"><div style="clear:both;padding-top:3px;">* "SGS" found in file<

In [16]:
def cleanRawRecords(rawRecords):
    cleanRecords = []

    for record in rawRecords:
        name = record['Main Info'].find('b').get_text()
        address = record['Main Info'].get_text()[record['Main Info'].get_text().find(name, len(name))+len(name):record['Main Info'].get_text().find('Tel.')]
        activity = record['Main Info'].find_all('font')[-1].get_text()
        for file in record['Files']:
            allThemes = file.find_all('div', class_=['theme1', 'theme2', 'theme3', 'theme4'])
            scope = ''
            subScope1 = ''
            subScope2 = ''
            subScope3 = ''
            for div in allThemes:
                if div.get('class') == ['theme1']:
                    scope = div.get_text()
                elif div.get('class') == ['theme2']:
                    subScope1 = div.get_text()
                elif div.get('class') == ['theme3']:
                    subScope2 = div.get_text()
                elif div.get('class') == ['theme4']:
                    subScope3 = div.get_text()
                    cleanRecords.append({'Name': name,
                                         'Address': address,
                                         'Activity': activity,
                                         'Scope': scope,
                                         'Sub-scope 1': subScope1,
                                         'Sub-scope 2': subScope2,
                                         'Sub-scope 3': subScope3})
    
    return cleanRecords

cleanRawRecords(getRawRecords(links[2]))

[{'Name': 'TÜV SÜD France',
  'Address': '\tFile : 1-5683 \nTÜV SÜD France\r\n\tLE NORLY I 42 CHE DU MOULIN CARRON | 69130 ECULLY | FRANCE\r\n\t',
  'Activity': '\r\n\tTesting',
  'Scope': 'ENVIRONMENT',
  'Sub-scope 1': 'ASBESTOS',
  'Sub-scope 2': 'Sampling',
  'Sub-scope 3': ''}]

In [17]:
bigListCleanRecords = []

for link in links:
    bigListCleanRecords.extend(cleanRawRecords(getRawRecords(link)))

bigListCleanRecords

[{'Name': 'AO SGS Vostok Limited',
  'Address': '\t\t\nAO SGS Vostok Limited\n\tBuilding 7 18 Andropova Prospekt |  MOSCOW  115432 | RUSSIAN FEDERATION\n\t',
  'Activity': '\n\tInspection',
  'Scope': 'AGRI-FOOD',
  'Sub-scope 1': 'Vegetal primary production',
  'Sub-scope 2': 'Qualitative and quantitative controls on agricultural products and their derivatives in the framework of commercial exchanges',
  'Sub-scope 3': 'Qualitative and quantitative control of bulk solids'},
 {'Name': 'AO SGS Vostok Limited',
  'Address': '\t\t\nAO SGS Vostok Limited\n\tBuilding 7 18 Andropova Prospekt |  MOSCOW  115432 | RUSSIAN FEDERATION\n\t',
  'Activity': '\n\tInspection',
  'Scope': 'AGRI-FOOD',
  'Sub-scope 1': 'Vegetal primary production',
  'Sub-scope 2': 'Qualitative and quantitative controls on agricultural products and their derivatives in the framework of commercial exchanges',
  'Sub-scope 3': 'Qualitative and quantitative control of packaged products (bags, drums, individual packaging ..

In [18]:
df = pandas.DataFrame(bigListCleanRecords)

df.head(5)

,Name,Address,Activity,Scope,Sub-scope 1,Sub-scope 2,Sub-scope 3
0,AO SGS Vostok Limited,\t\t\nAO SGS Vostok Limited\n\tBuilding 7 18 A...,\n\tInspection,AGRI-FOOD,Vegetal primary production,Qualitative and quantitative controls on agric...,Qualitative and quantitative control of bulk s...
1,AO SGS Vostok Limited,\t\t\nAO SGS Vostok Limited\n\tBuilding 7 18 A...,\n\tInspection,AGRI-FOOD,Vegetal primary production,Qualitative and quantitative controls on agric...,Qualitative and quantitative control of packag...
2,SGS AUTOMOTIVE SERVICES,\n\t1 PLACE DU GUÉ DE MAULNY | 72000 LE MANS\n\t,\n\tInspection,LEGAL METROLOGY,Transport - Security,[18.4.2] Prestations d'organisme désigné pour ...,Vérification d'installations des cinémomètres ...
3,SGS AUTOMOTIVE SERVICES,\n\t1 PLACE DU GUÉ DE MAULNY | 72000 LE MANS\n\t,\n\tInspection,LEGAL METROLOGY,Transport - Security,[18.4.2] Prestations d'organisme désigné pour ...,Vérification périodique des cinémomètres de co...
4,SGS AUTOMOTIVE SERVICES,\n\t1 PLACE DU GUÉ DE MAULNY | 72000 LE MANS\n\t,\n\tInspection,LEGAL METROLOGY,Transport - Security,[18.4.2] Prestations d'organisme désigné pour ...,Vérification périodique des cinémomètres de co...


In [19]:
df.to_csv('CoFrac Excel.csv')